# CREAR UNA BASE DE DATOS EN HIVE METAESTORE USANDO PYSPARK

1. CREAR UNA SPARKSESION

In [1]:
# Importar configuraciones

import sys
import os
sys.path.append(os.path.abspath('./src'))

from config.spark_config import SparkSession
from etl.transformaciones import TransformacionesVentas
from pyspark.sql import functions as F
from config.hdfs_uploader import subir_a_hdfs

In [2]:
spark = SparkSession.getActiveSession()
if spark:
    spark.stop()

# Inicializar Spark
spark = SparkSession.builder \
    .appName("CrearInsertarDatosSparkHive-Notebook") \
    .enableHiveSupport() \
    .getOrCreate()

26/02/12 23:12:22 WARN Utils: Your hostname, LenovoLOQ resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/02/12 23:12:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/12 23:12:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/12 23:12:24 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


2. CREAR LA BASE DE DATOS

In [3]:
spark.sql("CREATE DATABASE IF NOT EXISTS pyspark_hive")
spark.sql("SHOW DATABASES").show()

+------------+
|   namespace|
+------------+
|     default|
|pyspark_hive|
+------------+



3. CREAR LA TABLA EN LA BASE DE DATOS

In [4]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS pyspark_hive.empleado (
        ID INT,
        NOMBRE STRING,
        TELEFONO STRING,
        CORREO STRING,
        FECHA_INGRESO DATE,
        EDAD INT,
        SALARIO DOUBLE,
        ID_EMPRESA INT
    )
    STORED AS PARQUET
""")

DataFrame[]

In [5]:
spark.sql("""SHOW TABLES IN pyspark_hive;""").show()

+------------+---------+-----------+
|   namespace|tableName|isTemporary|
+------------+---------+-----------+
|pyspark_hive| empleado|      false|
+------------+---------+-----------+



4. CARGA DE DATOS - FLUJO CORRECTO

    Subir el CSV de LOCAL → HDFS
    Leer desde HDFS con PySpark
    Limpiar / convertir tipos
    Insertar en tabla Hive

```
hdfs dfs -mkdir -p /dataset/pyspark
hdfs dfs -put -f /home/hadoop/bigdata-spark2/dataset/empleado.csv /data/pyspark/
hdfs dfs -ls /data/pyspark
```

In [ ]:
#Usando la librare import os
# Ejecutar comando hdfs
os.system("hdfs dfs -mkdir -p /dataset/pyspark")
os.system("hdfs dfs -put -f /home/hadoop/bigdata-spark2/dataset/empleado.csv /data/pyspark/")
os.system("hdfs dfs -ls /data/pyspark")

Found 5 items
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 10:37 /data/pyspark/20260212_103727_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 10:38 /data/pyspark/20260212_103807_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 10:40 /data/pyspark/20260212_104037_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 10:41 /data/pyspark/20260212_104101_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 23:12 /data/pyspark/empleado.csv


0

In [ ]:
spark._jvm.org.apache.hadoop.fs.FileSystem \
    .get(spark._jsc.hadoopConfiguration()) \
    .copyFromLocalFile(
        False,
        True,
        spark._jvm.org.apache.hadoop.fs.Path("file:///home/hadoop/bigdata-spark2/dataset/empleado.csv"),
        spark._jvm.org.apache.hadoop.fs.Path("/data/pyspark/empleado.csv")
    )

In [ ]:
# Insertar en la caja de texto dataset/empleados.csv

import subprocess
from datetime import datetime

# 📌 Pedir ruta del archivo al usuario
local_path = input("📂 Ingresa la ruta completa del archivo a subir: ").strip()

# Verificar que exista
if not os.path.isfile(local_path):
    print("❌ El archivo no existe.")
    exit()

# Extraer nombre base
file_name = os.path.basename(local_path)

# Agregar timestamp para nombre automático
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
new_name = f"{timestamp}_{file_name}"

# Ruta destino en HDFS
hdfs_dir = "/data/pyspark"
hdfs_path = f"{hdfs_dir}/{new_name}"

print(f"🚀 Subiendo archivo como: {new_name}")

# Crear carpeta en HDFS si no existe
subprocess.run(["hdfs", "dfs", "-mkdir", "-p", hdfs_dir])

# Subir archivo
subprocess.run(["hdfs", "dfs", "-put", "-f", local_path, hdfs_path])

# Listar contenido
subprocess.run(["hdfs", "dfs", "-ls", hdfs_dir])

print("✅ Archivo cargado correctamente en HDFS")

In [ ]:
# Subir archivo y obtener ruta final en HDFS
hdfs_file = subir_a_hdfs("/home/hadoop/bigdata-spark2/dataset/empleado.csv")

print(f"Archivo disponible en HDFS en: {hdfs_file}")
hdfs_file

🚀 Subiendo archivo como: 20260212_231308_empleado.csv
Found 6 items
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 10:37 /data/pyspark/20260212_103727_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 10:38 /data/pyspark/20260212_103807_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 10:40 /data/pyspark/20260212_104037_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 10:41 /data/pyspark/20260212_104101_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 23:13 /data/pyspark/20260212_231308_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 23:13 /data/pyspark/empleado.csv
✅ Archivo cargado correctamente en HDFS: /data/pyspark/20260212_231308_empleado.csv
Archivo disponible en HDFS en: /data/pyspark/20260212_231308_empleado.csv


'/data/pyspark/20260212_231308_empleado.csv'

In [10]:
# Ingresar dataset/empleado.csv

hdfs_file = subir_a_hdfs()

🚀 Subiendo archivo como: 20260212_231343_empleado.csv
Found 7 items
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 10:37 /data/pyspark/20260212_103727_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 10:38 /data/pyspark/20260212_103807_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 10:40 /data/pyspark/20260212_104037_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 10:41 /data/pyspark/20260212_104101_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 23:13 /data/pyspark/20260212_231308_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 23:13 /data/pyspark/20260212_231343_empleado.csv
-rw-r--r--   1 hadoop supergroup      86917 2026-02-12 23:13 /data/pyspark/empleado.csv
✅ Archivo cargado correctamente en HDFS: /data/pyspark/20260212_231343_empleado.csv


4.2 Leer el CSV desde HDFS en PySpark

In [11]:
# Pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *

#ruta_hdfs = "hdfs:///data/pyspark/empleado.csv"
ruta_hdfs = hdfs_file

schema_raw = StructType([
    StructField("ID", StringType(), True),
    StructField("NOMBRE", StringType(), True),
    StructField("TELEFONO", StringType(), True),
    StructField("CORREO", StringType(), True),
    StructField("FECHA_INGRESO", StringType(), True),
    StructField("EDAD", StringType(), True),
    StructField("SALARIO", StringType(), True),
    StructField("ID_EMPRESA", StringType(), True)
])

df = spark.read \
    .option("header", "true") \
    .option("delimiter", ",") \
    .schema(schema_raw) \
    .csv(ruta_hdfs)

4.3 Limpiar y convertir datos (CLAVE)

In [12]:
# Pyspark
df_limpio = df \
    .withColumn("ID", col("ID").cast("int")) \
    .withColumn("EDAD", col("EDAD").cast("int")) \
    .withColumn("SALARIO", col("SALARIO").cast("double")) \
    .withColumn("ID_EMPRESA", col("ID_EMPRESA").cast("int")) \
    .withColumn(
        "FECHA_INGRESO",
        coalesce(
            to_date(col("FECHA_INGRESO"), "yyyy-MM-dd"),
            to_date(col("FECHA_INGRESO"), "dd/MM/yyyy")
        )
    ) \
    .withColumn(
        "CORREO",
        when(trim(col("CORREO")).isin("", "NA", "N/A"), None)
        .otherwise(col("CORREO"))
    )

# Opcional (recomendado):

df_limpio = df_limpio.filter(col("ID").isNotNull())

4.4 Verifica esquema antes de insertar

In [13]:
# Pyspark
df_limpio.printSchema()
df_limpio.show(5, truncate=False)

root
 |-- ID: integer (nullable = true)
 |-- NOMBRE: string (nullable = true)
 |-- TELEFONO: string (nullable = true)
 |-- CORREO: string (nullable = true)
 |-- FECHA_INGRESO: date (nullable = true)
 |-- EDAD: integer (nullable = true)
 |-- SALARIO: double (nullable = true)
 |-- ID_EMPRESA: integer (nullable = true)



+---+-------------+-----------------+------------------------+-------------+----+-------+----------+
|ID |NOMBRE       |TELEFONO         |CORREO                  |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+-------------+-----------------+------------------------+-------------+----+-------+----------+
|353|Mateo Mendoza|+56 89 7366-3177 |mateo.mendozalive.com   |2014-11-23   |29  |55560.0|6         |
|690|Bert Torres  |+507 73 8501-1586|bert.torres@terra.cl    |2019-10-19   |23  |13196.0|3         |
|486|Jonah Vera   |+593 903-986-474 |jonah.vera@uol.com.br   |2020-11-27   |0   |23455.0|6         |
|389|Jonah Rojas  |+593 25 6426-6907|jonah.rojas@gmail.com   |2016-03-17   |65  |20797.0|9         |
|32 |Carl Martinez|+502 46 1534-4792|carl.martinez@outlook.pe|2012-12-03   |63  |10764.0|8         |
+---+-------------+-----------------+------------------------+-------------+----+-------+----------+
only showing top 5 rows



4.5 Insertar en la tabla Hive pyspark_hive.empleado

In [14]:
# Pyspark
df_limpio.write \
    .mode("append") \
    .insertInto("pyspark_hive.empleado")

4.6 Validar carga

In [15]:
# Pyspark
df = spark.sql("SELECT * FROM pyspark_hive.empleado")
df.show(5, truncate=False)
     


+---+-------------+-----------------+------------------------+-------------+----+-------+----------+
|ID |NOMBRE       |TELEFONO         |CORREO                  |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+-------------+-----------------+------------------------+-------------+----+-------+----------+
|353|Mateo Mendoza|+56 89 7366-3177 |mateo.mendozalive.com   |2014-11-23   |29  |55560.0|6         |
|690|Bert Torres  |+507 73 8501-1586|bert.torres@terra.cl    |2019-10-19   |23  |13196.0|3         |
|486|Jonah Vera   |+593 903-986-474 |jonah.vera@uol.com.br   |2020-11-27   |0   |23455.0|6         |
|389|Jonah Rojas  |+593 25 6426-6907|jonah.rojas@gmail.com   |2016-03-17   |65  |20797.0|9         |
|32 |Carl Martinez|+502 46 1534-4792|carl.martinez@outlook.pe|2012-12-03   |63  |10764.0|8         |
+---+-------------+-----------------+------------------------+-------------+----+-------+----------+
only showing top 5 rows



In [16]:
# Detener Spark
from config.spark_config import detener_spark_session
detener_spark_session(spark)

🔴 Sesión de Spark detenida
